# Cliente-Automatización del Sistema

Autor: David U. Ordiz.

**Descripción del Notebook:**

En el anterior notebook hemos visto cómo crear un servidor de datos de ticks y cómo almacenar dicha información en una base de datos local constantemente actualizada. En este notebook vamos a ver cómo acceder a estos datos, cómo procesarlos para generar señales de compra y venta y cómo enrutar órdenes a la API de IB.

*Importante:* El <u>2-Notebook IB-Servidor de Datos de Ticks</u> se debe estar ejecutando y la TWS abierta.

<u>Reglas de la estrategia:</u>

* <u>Abre largos:</u> Close[0] > MMS(5)[0]
* <u>Cierra largos:</u> Close[0] < MMS(5)[0]

**Puntos que veremos en este Notebook:**

1. Importamos las librerías necesarias.
2. Cómo conectarmos a nuestra base de datos SQL, y obtener la última actualización de datos.
3. Cómo crear un bucle que nos permita obtener datos actualizados de la base de datos.
4. Cómo enrutar ordenes de compra/venta a través de la API de IB.
5. Creamos un gráfico interactivo usando la librería Plotly. Permite ver la evolución del precio de cierre respecto a la MMS (5).
6. Juntando todas las piezas en un único bloque de código: 
        6.1 Obtenemos datos de la base de datos en vivo.
        6.2 Calculamos la MMS y calculamos la posición de la estrategia.
        6.3 Asignamos valores a los plots del gráfico de Plotly.
        6.4 Procesamos las señales de compra y venta y ejecutamos las órdenes a través de la API de IB.
        6.5 Imprimos en pantalla la posición actual de la estrategia y las últimas columnas del DataFrame utilizado.

####  1. Importamos las librerías necesarias

In [ ]:
import sqlalchemy
import pandas as pd
import numpy as np
import plotly 
import time
from IPython.display import clear_output
from datetime import datetime
from ib_insync import *
# Inicia un loop para mantener el notebook continuamente actualizado. 
util.startLoop()
print("Librerías importadas correctamente")

Librerías importadas correctamente


#### 2. Nos conectamos a nuestra base de datos SQL, y obtenemos la última actualización de datos.

In [22]:
# Crea el motor para la base de datos.
engine = sqlalchemy.create_engine('sqlite:///BaseDeDatosIB.db')
# Creamos un dataframe y a través de la función read_sql() obtenemos los 
# datos de la tabla EURUSD. Esto nos da una foto fija (Snapshoot) con 
# la última actualización de los datos de la tabla.
df = pd.read_sql('EURUSD', engine, index_col='Time')
print(df.tail())

                            LastPrice
Time                                 
2024-11-22 12:21:41.208871   1.042145
2024-11-22 12:21:41.408940   1.042120
2024-11-22 12:21:47.516860   1.042095
2024-11-22 12:21:47.777937   1.042090
2024-11-22 12:21:51.202035   1.042085


#### 3. Cómo crear un loop que nos permita obtener datos actualizados de la base de datos.

In [23]:
while True:    
    clear_output(wait=True)
    
    # Crea una variable para almacenar nuestra Query.
    # Esta 'Query' obtiene los últimos 100 ticks de nuestra base de datos.
    Query = 'SELECT * FROM EURUSD ORDER BY Time DESC LIMIT 100'
    
     # Solicitamos los datos a la base de datos.
    df = pd.read_sql(Query, engine, index_col='Time')
    
    # Convierte el índice en formato datetime (necesario para df.resample())
    df.index = pd.to_datetime(df.index)
    
    # Convierte la serie de ticks en una serie OHLC en time frame de 5 segundos.
    df = df['LastPrice'].resample('5s').ohlc()
    
    # Elimina las filas que contienen valores nulos.
    df.dropna(inplace=True )
    
    # Muestra el dataframe actualizado
    print(df.tail(10))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S.%f")
    print("Ultima actualización: ", current_time)

KeyboardInterrupt: 

#### 4. Cómo enrutar órdenes de compra y venta a través de la API de IB.

In [24]:
# Crea un objeto IB
ib = IB()
# Nos conectamos a IB
ib.connect('127.0.0.1', 7497, clientId = 15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [25]:
# Crea y envía una orden de compra. 
order = MarketOrder('BUY', 500)
Contract = Forex('EURUSD')
trade = ib.placeOrder(Contract, order)

In [26]:
# Crea y envía una orden de venta. 
order = MarketOrder('SELL', 500)
Contract = Forex('EURUSD')
trade = ib.placeOrder(Contract, order)

#### 5. Creamos un gráfico interactivo usando la librería Plotly.

In [ ]:
# Importamos Plotly.graph_objects. Esta librería permite crear una
# figura interactiva que mostrará la evolución del precio de cierre
# y la evolución de la media móvil. 
import plotly.graph_objects as go

# Creamos la figura.
fig = go.FigureWidget(layout_title_text="EURUSD SMA Strategy")
# A través de add_scatter añadimos los dos plots que necesitamos.
fig.add_scatter(name = 'Close', line = dict(width = 1, color = 'black'))
fig.add_scatter(name = 'MMS(5)', line = dict(width = 1))

# Añadimos las etiquetas as los ejes Y y X.
fig['layout']['yaxis']['title']='Escala de Precios'
fig['layout']['xaxis']['title']='Fecha'
fig

FigureWidget({
    'data': [{'line': {'color': 'black', 'width': 1},
              'name': 'Close',
              'type': 'scatter',
              'uid': '2acaaa25-35c7-4ddb-b0b2-ac177b614a4d'},
             {'line': {'width': 1}, 'name': 'MMS(5)', 'type': 'scatter', 'uid': 'c22eee43-7bce-413d-861f-2b1930767bc0'}],
    'layout': {'template': '...',
               'title': {'text': 'EURUSD SMA Strategy'},
               'xaxis': {'title': {'text': 'Fecha'}},
               'yaxis': {'title': {'text': 'Escala de Precios'}}}
})

#### 6. Juntando todas las piezas en un único bloque de código: 
        6.1 Obtenemos datos de la base de datos en vivo.
        6.2 Calculamos la MMS y calculamos la posición de la estrategia.
        6.3 Asignamos valores a los plots del gráfico de Plotly.
        6.4 Procesamos las señales de compra y venta y ejecutamos las órdenes a través de la API de IB.
        6.5 Imprimos en pantalla la posición actual de la estrategia y las últimas columnas del DataFrame utilizado.

In [ ]:
# Esta variable almacenará la posición de la estrategia.
# La iniciamos con el valor 'Flat'
PosicionStrategia = 'Flat'

import datetime
# La variable LastTimeBar va a almacenar lo hora o timestamp
# de la última barra de nuestro DataFrame. La utilizaremos para 
# identificar el primer tick de una nueva barra. Este modelo 
# ejecuta las señales de compra y venta cuando ocurre el primer 
# tick de una barra. 
# Iniciamos la variable con un valor alto.
LastTimeBar = datetime.datetime(2150, 5, 17)

# El bucle while se ejecuta de forma contínua. 
while True: 
    # Limpia el último resultado mostrado.
    clear_output(wait=True)
    
    # 1.OBTENEMOS DATOS DE LA BASE DE DATOS 
    # ======================================================================
    
    # Esta 'Query' obtiene los últimos 300 ticks de nuestra base de datos.
    Query = 'SELECT * FROM EURUSD ORDER BY Time DESC LIMIT 300'    
    
    # Solicitamos los datos a la base de datos.
    df = pd.read_sql(Query, engine, index_col='Time')   
    
    # Convierte el índice en formato datetime (necesario para df.resample())
    df.index = pd.to_datetime(df.index)
    
    # Convierte la serie de ticks en una serie OHLC en time frame de 5s.
    df = df['LastPrice'].resample('5s').ohlc()
    
    # Elimina las filas que contienen valores nulos.
    df.dropna(inplace = True )
    
    # 2.CÁLCULO DE LA MMS Y POSICIÓN DE LA ESTRATEGIA.
    # ======================================================================
    
    # Crea la media móvil simple (MMS)
    df['MMS'] = df.close.rolling(5).mean()
    # Elimina los valores nulos.
    df.dropna(inplace=True )
    # Crea la columa posición.
    df['Posicion'] = np.where(df['close'] > df['MMS'], 1, 0)
    # Limpia el dataframe.
    df = df[['close','MMS','Posicion']]
    
    # 3.ASIGNACIÓN DE VALORES AL GRÁFICO.
    # ======================================================================
    
    # Actualiza el gráfico cada vez que hay una pasada en el bucle
    # Fija los valores de X e Y para los dos plots de la figura.
    fig.data[0].x = df.index
    fig.data[0].y = df['close']
    fig.data[1].x = df.index
    fig.data[1].y = df['MMS']
    
    # 4.PROCESAMIENTO DE SEÑALES Y EJECUCIÓN DE ÓRDENES.
    # ======================================================================
    
    # Para abrir una una posición se deben satisfacer cuatro condiciones.
    # 1) Una nueva barra se acaba de iniciar. La hora de la barra actual es 
    #    mayor que la hora de la variable LastTimeBar.
    # 2) La posición de la estrategia en la penúltima barra es 1 (Long).
    #    Nota: Utilizamos la penúltima barra (iloc[-2]) ya que la última 
    #    barra (iloc[-1]) representa la barra en la que se ha generado el 
    #    nuevo tick. Por lo tanto sería erróneo. Aquí queremos abrir una 
    #    posición en la apertura de la barra actual si se ha producido una 
    #    señal de compra en la barra anterior. Por ello se usa la penúltima 
    #    barra.
    # 3) La posición en la antepenúltima barra es 0 (Flat).
    # 4) El valor de la variable PosicionStrategia debe ser 'Flat'.
    
    if (df.index[-1] > LastTimeBar and df.Posicion.iloc[-2] == 1 and 
        df.Posicion.iloc[-3] == 0 and PosicionStrategia == 'Flat'):
        
        # Crea y envía una order de compra.
        order = MarketOrder('BUY', 100)
        trade = ib.placeOrder(Contract, order)
        PosicionStrategia = 'Long' 
        
    # Cierra posiciones largas cuando la penúltima barra es 0 y el valor
    # de la variable PosicionStrategia es 'Long'
    if (df.Posicion.iloc[-2] == 0 and PosicionStrategia == 'Long'):
        # Crea y envía una order de venta.
        order = MarketOrder('SELL', 100)
        trade = ib.placeOrder(Contract, order)
        PosicionStrategia = 'Flat'
    
    # Almacena la hora de la última barra.
    LastTimeBar = df.index[-1]
    
    # 5.IMPRIME ÚLTIMAS COLUMNAS DEL DATAFRAME Y LA POSICIÓN DEL MODELO.
    # ======================================================================
    
    # Muestra el dataframe actualizado
    print(df.tail(5))
    print('La posición de la estrategia es: ', PosicionStrategia)
    
    HoraActual = datetime.datetime.now()
    HoraFormateada = HoraActual.strftime('%H:%M:%S.%f')
    print("Ultima actualización: ", HoraFormateada)   

                        close       MMS  Posicion
Time                                             
2023-05-31 18:17:15  1.064325  1.064402         0
2023-05-31 18:17:20  1.064225  1.064364         0
2023-05-31 18:17:25  1.064255  1.064336         0
2023-05-31 18:17:30  1.064200  1.064284         0
2023-05-31 18:17:35  1.064180  1.064237         0
La posición de la estrategia es:  Flat
Ultima actualización:  18:17:36.677597
